In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
file_SIM = '../Data/Sistema de Informação sobre Mortalidade – SIM/DO23OPEN.csv'

In [4]:
data_SIM = pd.read_csv(file_SIM, sep=';', low_memory=False)

In [5]:
for col in data_SIM.columns:
    tipos = data_SIM[col].apply(type).nunique()
    if tipos > 1:
        print(f"⚠️ Coluna '{col}' tem múltiplos tipos de dados. Convertendo para string.")
        data_SIM[col] = data_SIM[col].astype(str)

⚠️ Coluna 'LINHAA' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAB' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAC' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAD' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'LINHAII' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'STCODIFICA' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CODIFICADO' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'VERSAOSIST' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CAUSAMAT' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'FONTES' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'TPNIVELINV' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CAUSABAS_O' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'TPPOS' tem múltiplos tipos de dados. Convertendo para string.
⚠️ Coluna 'CB_AL

In [6]:
data_SIM.head()

,contador,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,ESC2010,SERIESCFAL,OCUP,CODMUNRES,LOCOCOR,CODESTAB,CODMUNOCOR,IDADEMAE,ESCMAE,ESCMAE2010,SERIESCMAE,OCUPMAE,QTDFILVIVO,QTDFILMORT,GRAVIDEZ,SEMAGESTAC,GESTACAO,PARTO,OBITOPARTO,PESO,TPMORTEOCO,OBITOGRAV,OBITOPUERP,ASSISTMED,EXAME,CIRURGIA,NECROPSIA,LINHAA,LINHAB,LINHAC,LINHAD,LINHAII,CAUSABAS,CB_PRE,COMUNSVOIM,DTATESTADO,CIRCOBITO,ACIDTRAB,FONTE,NUMEROLOTE,DTINVESTIG,DTCADASTRO,ATESTANTE,STCODIFICA,CODIFICADO,VERSAOSIST,VERSAOSCB,FONTEINV,DTRECEBIM,ATESTADO,DTRECORIGA,OPOR_DO,CAUSAMAT,ESCMAEAGR1,ESCFALAGR1,STDOEPIDEM,STDONOVA,DIFDATA,NUDIASOBCO,DTCADINV,TPOBITOCOR,DTCONINV,FONTES,TPRESGINFO,TPNIVELINV,DTCADINF,MORTEPARTO,DTCONCASO,ALTCAUSA,CAUSABAS_O,TPPOS,TP_ALTERA,CB_ALT
0,1,1,2,14022023,930.0,833.0,330270.0,10071954.0,468,1,4.0,1.0,2.0,1.0,NaN,715210.0,330190,3,NaN,330190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,2.0,*R99X,*I10X,nan,nan,nan,I10,NaN,NaN,14022023.0,NaN,NaN,NaN,20230018.0,NaN,23022023.0,5.0,S,S,3.2.30,3.4,NaN,27022023.0,R99/I10,27022023,13,nan,NaN,10.0,0.0,1,13,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,I10,nan,NaN,nan
1,2,1,2,14022023,1730.0,822.0,221110.0,12091956.0,466,1,4.0,2.0,4.0,3.0,NaN,21210.0,221100,1,2323281.0,221100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,*R578,*I619,*I678,nan,nan,I678,NaN,NaN,14022023.0,NaN,NaN,NaN,20230028.0,NaN,17022023.0,5.0,S,S,3.2.30,3.4,NaN,17022023.0,R578/I619/I678,17022023,3,nan,NaN,12.0,0.0,1,3,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,I678,nan,NaN,nan
2,3,1,2,14022023,500.0,822.0,220790.0,10011953.0,470,2,NaN,3.0,9.0,9.0,NaN,999992.0,220790,1,2726971.0,221100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,*A419,*K750,nan,nan,*E149*I10X,K750,NaN,NaN,14022023.0,NaN,NaN,NaN,20230028.0,NaN,17022023.0,5.0,S,S,3.2.30,3.4,NaN,17022023.0,A419/K750*E149 I10,17022023,3,nan,NaN,9.0,0.0,1,3,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,K750,nan,NaN,nan
3,4,1,2,14022023,2235.0,823.0,231180.0,15121943.0,479,2,1.0,2.0,4.0,3.0,NaN,763010.0,230440,1,2497654.0,230440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,*I611,nan,nan,nan,*J189,I611,NaN,NaN,15022023.0,NaN,NaN,NaN,20230007.0,23022023.0,15022023.0,1.0,S,S,3.2.30,NaN,3.0,9032023.0,I611*J189,2032023,16,nan,NaN,12.0,0.0,1,23,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,I64,S,NaN,nan
4,5,1,2,14022023,940.0,841.0,411350.0,9041973.0,449,2,4.0,1.0,1.0,0.0,NaN,999992.0,510515,2,7050577.0,510515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,2.0,3.0,1.0,NaN,NaN,2.0,*J960,*C189,nan,nan,nan,C189,NaN,NaN,14022023.0,NaN,NaN,NaN,20240057.0,14022023.0,14022023.0,1.0,S,S,3.2.30,3.4,3.0,30102024.0,J960/C189,16022023,2,nan,NaN,0.0,0.0,1,624,3.0,17022023.0,9.0,17022023.0,nan,NaN,M,NaN,NaN,NaN,NaN,C189,S,NaN,nan


In [7]:
data_SIM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465610 entries, 0 to 1465609
Data columns (total 86 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   contador    1465610 non-null  int64  
 1   ORIGEM      1465610 non-null  int64  
 2   TIPOBITO    1465610 non-null  int64  
 3   DTOBITO     1465610 non-null  int64  
 4   HORAOBITO   1416392 non-null  float64
 5   NATURAL     1399908 non-null  float64
 6   CODMUNNATU  1381624 non-null  float64
 7   DTNASC      1462816 non-null  float64
 8   IDADE       1465610 non-null  int64  
 9   SEXO        1465610 non-null  int64  
 10  RACACOR     1442526 non-null  float64
 11  ESTCIV      1399251 non-null  float64
 12  ESC         1375905 non-null  float64
 13  ESC2010     1367238 non-null  float64
 14  SERIESCFAL  404622 non-null   float64
 15  OCUP        1263542 non-null  float64
 16  CODMUNRES   1465610 non-null  int64  
 17  LOCOCOR     1465610 non-null  int64  
 18  CODESTAB    1064156 no

## Carga dos dados

### Conexão com banco

In [8]:
DB_USER = "postgres"
DB_PASSWORD = "root"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mortalidades"

In [9]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [10]:
with engine.connect() as conn:
    print("✅ Conectado ao banco mortalidade")

✅ Conectado ao banco mortalidade


In [11]:
def inserir_dados(df, tabela):
    df.to_sql(tabela, con=engine, if_exists="append", index=False)
    print(f"✅ Dados inseridos na tabela {tabela}")

### Inserção dos dados

In [12]:
obitos = data_SIM[['DTOBITO', 'HORAOBITO', 'CODMUNOCOR', 'CAUSABAS', 'ASSISTMED', 'NECROPSIA', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR']]
obitos

,DTOBITO,HORAOBITO,CODMUNOCOR,CAUSABAS,ASSISTMED,NECROPSIA,DTNASC,IDADE,SEXO,RACACOR
0,14022023,930.0,330190,I10,9.0,2.0,10071954.0,468,1,4.0
1,14022023,1730.0,221100,I678,1.0,2.0,12091956.0,466,1,4.0
2,14022023,500.0,221100,K750,1.0,2.0,10011953.0,470,2,NaN
3,14022023,2235.0,230440,I611,1.0,2.0,15121943.0,479,2,1.0
4,14022023,940.0,510515,C189,1.0,2.0,9041973.0,449,2,4.0
...,...,...,...,...,...,...,...,...,...,...
1465605,30112023,2100.0,160050,R99,2.0,2.0,1042023.0,307,1,5.0
1465606,12122023,1220.0,150060,R98,2.0,2.0,23101987.0,436,1,5.0
1465607,22122023,1734.0,150060,R98,2.0,2.0,5112023.0,301,1,5.0
1465608,22122023,2030.0,130140,X709,2.0,9.0,1012008.0,415,2,5.0


In [13]:
query = "SELECT * FROM localizacoes"
df_localizacoes = pd.read_sql(query, con=engine)

In [14]:
df_localizacoes.info()
df_localizacoes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_local   5570 non-null   int64 
 1   municipio  5570 non-null   object
 2   estado     5570 non-null   object
 3   regiao     5570 non-null   object
dtypes: int64(1), object(3)
memory usage: 174.2+ KB


,id_local,municipio,estado,regiao
0,110001,Alta Floresta D'Oeste,RO,Norte
1,110037,Alto Alegre dos Parecis,RO,Norte
2,110040,Alto Paraíso,RO,Norte
3,110034,Alvorada D'Oeste,RO,Norte
4,110002,Ariquemes,RO,Norte


In [15]:
query = "SELECT id_causa, codigo_cid, descricao FROM causas"
df_causas = pd.read_sql(query, con=engine)

In [16]:
df_causas.info()
df_localizacoes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5601 entries, 0 to 5600
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_causa    5601 non-null   int64 
 1   codigo_cid  5601 non-null   object
 2   descricao   5601 non-null   object
dtypes: int64(1), object(2)
memory usage: 131.4+ KB


,id_local,municipio,estado,regiao
0,110001,Alta Floresta D'Oeste,RO,Norte
1,110037,Alto Alegre dos Parecis,RO,Norte
2,110040,Alto Paraíso,RO,Norte
3,110034,Alvorada D'Oeste,RO,Norte
4,110002,Ariquemes,RO,Norte


In [17]:
obitos = obitos.merge(df_causas[['codigo_cid','id_causa']],
                     left_on='CAUSABAS',
                     right_on='codigo_cid',
                     how='left')

In [18]:
obitos.drop(columns=['codigo_cid'], inplace=True)

In [19]:
obitos.drop(columns=['HORAOBITO'], inplace=True)

In [20]:
obitos.rename(columns={
    'DTOBITO': 'data',
    'CODMUNOCOR': 'id_local',
    'CAUSABAS':'CID',
    'ASSISTMED':'assist_med',
    'NECROPSIA':'necropsia',
    'DTNASC':'data_nascimento',
    'IDADE':'idade',
    'SEXO':'sexo',
    'RACACOR':'id_raca_cor',
    'id_causa':'id_causa_principal'
}, inplace=True)

In [21]:
obitos['id_raca_cor'] = obitos['id_raca_cor'].fillna(9).astype('int64')

In [22]:
obitos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465610 entries, 0 to 1465609
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   data                1465610 non-null  int64  
 1   id_local            1465610 non-null  int64  
 2   CID                 1465610 non-null  object 
 3   assist_med          1020765 non-null  float64
 4   necropsia           1047269 non-null  float64
 5   data_nascimento     1462816 non-null  float64
 6   idade               1465610 non-null  int64  
 7   sexo                1465610 non-null  int64  
 8   id_raca_cor         1465610 non-null  int64  
 9   id_causa_principal  1465610 non-null  int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 111.8+ MB


In [23]:
obitos['data'] = obitos['data'].astype(str)

In [24]:
obitos['data'] = obitos['data'].str.zfill(8)

In [25]:
obitos['data'] = pd.to_datetime(obitos['data'], format = '%d%m%Y', errors='coerce')

In [26]:
obitos.head()

,data,id_local,CID,assist_med,necropsia,data_nascimento,idade,sexo,id_raca_cor,id_causa_principal
0,2023-02-14,330190,I10,9.0,2.0,10071954.0,468,1,4,1
1,2023-02-14,221100,I678,1.0,2.0,12091956.0,466,1,4,2
2,2023-02-14,221100,K750,1.0,2.0,10011953.0,470,2,9,3
3,2023-02-14,230440,I611,1.0,2.0,15121943.0,479,2,1,4
4,2023-02-14,510515,C189,1.0,2.0,9041973.0,449,2,4,5


In [27]:
def converter_idade(idade):
    idade_str = str(int(idade)).zfill(3)

    unidade = int(idade_str[0])
    quantidade = int(idade_str[1:])

    if unidade == 1:  # Minutos Converter para dias (~1 dia = 1440 minutos)
        return round(quantidade / 1440, 2)
    elif unidade == 2:  # Horas Converter para dias (~1 dia = 24 horas)
        return round(quantidade / 24, 2)
    elif unidade == 3:  # Dias Converter para anos (~365 dias)
        return round(quantidade / 365, 2)
    elif unidade == 4:  # Já está em anos
        return quantidade
    elif unidade == 5:  # Idade maior que 100 anos
        return 100 + quantidade
    else:  # Ignorado
        return np.nan

In [28]:
obitos['idade'] = obitos['idade'].apply(converter_idade)

In [29]:
obitos['data_nascimento'] = pd.to_datetime(obitos['data_nascimento'], format='%d%m%Y', errors='coerce')

In [30]:
obitos['idade'] = obitos['idade'].fillna(0).astype(int)

In [31]:
sexo_map = {1: 'M', 2: 'F', 9: 'I'}
obitos['sexo'] = obitos['sexo'].map(sexo_map).fillna('I')

In [32]:
assist_map = {1: 'sim', 2: 'não', 9: 'ignorado'}
obitos['necropsia'] = obitos['necropsia'].fillna(9).map(assist_map)

In [33]:
assist_map = {1: 'sim', 2: 'não', 9: 'ignorado'}
obitos['assist_med'] = obitos['assist_med'].fillna(9).map(assist_map)

In [34]:
obitos['id_raca_cor'] = obitos['id_raca_cor'].fillna(9).astype(int)

In [35]:
print(obitos.dtypes)

data                  datetime64[ns]
id_local                       int64
CID                           object
assist_med                    object
necropsia                     object
data_nascimento       datetime64[ns]
idade                          int32
sexo                          object
id_raca_cor                    int32
id_causa_principal             int64
dtype: object


In [36]:
obitos.drop(columns=['CID'], inplace=True)

In [37]:
obitos.head()

,data,id_local,assist_med,necropsia,data_nascimento,idade,sexo,id_raca_cor,id_causa_principal
0,2023-02-14,330190,ignorado,não,1954-07-10,68,M,4,1
1,2023-02-14,221100,sim,não,1956-09-12,66,M,4,2
2,2023-02-14,221100,sim,não,1953-01-10,70,F,9,3
3,2023-02-14,230440,sim,não,1943-12-15,79,F,1,4
4,2023-02-14,510515,sim,não,1973-04-09,49,F,4,5


In [38]:
query = "SELECT DISTINCT id_local FROM localizacoes"
df_localizacoes = pd.read_sql(query, con=engine)
id_locais_existentes = set(df_localizacoes["id_local"])

In [39]:
id_locais_obitos = set(obitos["id_local"])
id_faltantes = id_locais_obitos - id_locais_existentes 

In [40]:
if id_faltantes:
    print(f"🚨 Os seguintes id_local não existem na tabela localizacoes e foram ignorados:")
    print(id_faltantes)

🚨 Os seguintes id_local não existem na tabela localizacoes e foram ignorados:
{330000, 510000, 210000, 430000, 310000, 350000, 290000}


In [41]:
obitos_filtrado = obitos[obitos["id_local"].isin(id_locais_existentes)]

In [42]:
if not obitos_filtrado.empty:
    obitos_filtrado.to_sql("obitos", con=engine, if_exists="append", index=False)
    print("✅ Dados inseridos com sucesso na tabela obitos!")
else:
    print("⚠️ Nenhum dado válido para inserção!")

✅ Dados inseridos com sucesso na tabela obitos!
